## ETL DEL DATASET PRINCIPAL: MOVIES

In [4]:
#Es necesario importar las depencendias necesarias
import pandas as pd
from datetime import date, datetime

In [6]:
# Importamos el dataset de trabajo
df = pd.read_csv('./dataset/movies_dataset.csv', delimiter=',')
# Se elimina las columnas que no serán utilizadas, video,imdb_id,adult,original_title,poster_path y homepage
df = df.drop(columns = ['video','imdb_id','adult','original_title','poster_path','homepage']) 
# df.head()
print(" El dataset tiene", df.shape[0],"filas. ")

 El dataset tiene 45466 filas. 


C:\Users\HP\AppData\Local\Temp\ipykernel_11752\369045348.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./dataset/movies_dataset.csv', delimiter=',')


In [7]:
# Metodo para detectar los datos duplicados
# me sirve para ver si existen registros duplicados
df.duplicated().sum()
# Se averigua cuantos registros ("diferentes") duplicados hay.

13

In [8]:
# Se eliminan esos registros duplicados:

df = df.drop_duplicates()
# datafrane sin duplicados
df.shape

print(" El dataset queda con ", df.shape[0],"filas. ")

 El dataset queda con  45453 filas. 


In [9]:
# Se averigua cuantos valores nulos se tiene por columna:
df.isnull().sum()

print("Los campos que poseen nulos son: \n",df.isnull().sum())

Los campos que poseen nulos son: 
 belongs_to_collection    40959
budget                       0
genres                       0
id                           0
original_language           11
overview                   954
popularity                   5
production_companies         3
production_countries         3
release_date                87
revenue                      6
runtime                    263
spoken_languages             6
status                      87
tagline                  25045
title                        6
vote_average                 6
vote_count                   6
dtype: int64


In [10]:
# COLUMNA revenue
# Imputamos con el valor 0 los campos nulos de la con una revenue - recaudacion de la pelicula

df['revenue'].fillna(0, inplace=True)

df.isnull().sum()

belongs_to_collection    40959
budget                       0
genres                       0
id                           0
original_language           11
overview                   954
popularity                   5
production_companies         3
production_countries         3
release_date                87
revenue                      0
runtime                    263
spoken_languages             6
status                      87
tagline                  25045
title                        6
vote_average                 6
vote_count                   6
dtype: int64

In [11]:
# Se averigua el tipo de dato de cada columna y se decide si es el tipo de dato correcto
# Asimismo al corregir el tipo de dato, se analizan los datos cargados. budge es el presupuesto de la pelicula pasar a float
# Pero primero remplazamos otros valores.
df['budget'].replace('/ff9qCepilowshEtG2GYWwzt2bs4.jpg', 0, inplace = True)
df['budget'].replace('/zV8bHuSL6WXoD6FWogP9j4x80bL.jpg', 0, inplace = True)
df['budget'].replace('/zaSf5OG7V8X8gqFvly88zDdRm46.jpg', 0, inplace = True)

In [12]:
# VEEEEER id 

df['id'].duplicated().sum()
# Se averigua cuantos registros ("diferentes") duplicados hay. ver estooooo

17

In [13]:
# Los valores nulos del campo release date (fecha de estreno) deben eliminarse. 
# Se decide eliminar pues no serviran para las consultas en la API posteriormente.

df['release_date'].replace('1',None, inplace = True)
df['release_date'].replace('12',None, inplace = True)
df['release_date'].replace('22',None, inplace = True)
df.dropna(subset='release_date',inplace=True )

df.isnull().sum()


belongs_to_collection    40875
budget                       0
genres                       0
id                           0
original_language           11
overview                   941
popularity                   0
production_companies         0
production_countries         0
release_date                 0
revenue                      0
runtime                    246
spoken_languages             0
status                      80
tagline                  24969
title                        0
vote_average                 0
vote_count                   0
dtype: int64

In [14]:
print(" El dataset tiene", df.shape[0],"filas. ")

 El dataset tiene 45363 filas. 


In [15]:
df['release_date'].sort_values(ascending=True)

34940    1874-12-09
34937    1878-06-14
41602    1883-11-19
34933    1887-08-18
34934    1888-01-01
            ...    
44535    2018-04-04
38130    2018-04-25
30402    2018-11-07
38885    2018-12-31
26559    2020-12-16
Name: release_date, Length: 45363, dtype: object

In [16]:
# De haber fechas, deberán tener el formato AAAA-mm-dd, 
# además deberán crear la columna release_year donde extraerán el año de la fecha de estreno.
# Se crea tambien las columnas release_month y release_day_of_week para ser utilizadas en los metodos de la api.
df['release_date']          = pd.to_datetime(df['release_date'])
df['release_year']          = df['release_date'].dt.year 
df['release_month']         = df['release_date'].dt.month 
df['release_day_of_week']   = df['release_date'].dt.day_of_week

# verificar algunos registros
df.head()


,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,release_year,release_month,release_day_of_week
0,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,...,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,7.7,5415.0,1995,10,0
1,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,...,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,1995,12,4
2,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,en,A family wedding reignites the ancient feud be...,11.7129,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,...,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,1995,12,4
3,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,...,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,1995,12,4
4,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",11862,en,Just when George Banks has recovered from his ...,8.387519,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,...,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0,1995,2,4


In [17]:
""" Se crea la columna con el retorno de inversión, necesario para el funcionamiento de la api, 
llamada return con los campos revenue y budget, dividiendo estas dos últimas revenue / budget, 
cuando no hay datos disponibles para calcularlo, deberá tomar el valor 0. """

# se averigua el tipo de datos de las columnas se remplaza los puntos por coma, 
# antes de hacer la operacion df['revenue']/df['budget']
df['budget'] = df['budget'].astype(float)
df['revenue'] = df['revenue'].astype(float)

In [18]:
df['retorno'] = (df['revenue'].astype(float) / df['budget'].astype(float))
# validad que sean ambas variables mayor a cero

#df['retorno'] = df.where((df['revenue'].astype(float) > 0 & df['budget'].astype(float) > 0), (df['revenue'].astype(float) /df['budget'].astype(float) ), 0)
df['retorno']
#df['retorno'] = df['retorno'].apply(lambda x: (df['revenue']/df['budget']) if (df['revenue']>0 & df['budget']>0) else 0)


0        12.451801
1         4.043035
2              NaN
3         5.090760
4              inf
           ...    
45460          NaN
45462          NaN
45463          NaN
45464          NaN
45465          NaN
Name: retorno, Length: 45363, dtype: float64

In [19]:
df.to_csv('../movies/dataset/movies_limpio.csv', index=False)

## ETL DEL DATASET DE CREDITS

In [20]:
# Importamos el dataset de trabajo
df_c = pd.read_csv('./dataset/credits.csv', delimiter=',')
print(" El dataset tiene", df_c.shape[0],"filas. ")

 El dataset tiene 45476 filas. 


In [21]:
df_c

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862
...,...,...,...
45471,"[{'cast_id': 0, 'character': '', 'credit_id': ...","[{'credit_id': '5894a97d925141426c00818c', 'de...",439050
45472,"[{'cast_id': 1002, 'character': 'Sister Angela...","[{'credit_id': '52fe4af1c3a36847f81e9b15', 'de...",111109
45473,"[{'cast_id': 6, 'character': 'Emily Shaw', 'cr...","[{'credit_id': '52fe4776c3a368484e0c8387', 'de...",67758
45474,"[{'cast_id': 2, 'character': '', 'credit_id': ...","[{'credit_id': '533bccebc3a36844cf0011a7', 'de...",227506


In [22]:
# Metodo para detectar los datos duplicados
# me sirve para ver si existen registros duplicados
df_c.duplicated().sum()
# Se averigua cuantos registros ("diferentes") duplicados hay.

37

In [23]:
# Se eliminan esos registros duplicados:
df_c = df_c.drop_duplicates()
# datafrane sin duplicados
df_c.shape

print(" El dataset queda con ", df_c.shape[0],"filas. ")

 El dataset queda con  45439 filas. 


In [24]:
# Se averigua cuantos valores nulos se tiene por columna:
df_c.isnull().sum()

print("Los campos que poseen nulos son: \n",df_c.isnull().sum()) # no hay nulos 

Los campos que poseen nulos son: 
 cast    0
crew    0
id      0
dtype: int64


In [25]:
df_c.to_csv('../movies/dataset/credits_limpio.csv', index=False)

In [26]:
print('tipo de dato para movies', df['id'].dtype, 'y el tipo de dato para credits ',df_c['id'].dtype)

tipo de dato para movies object y el tipo de dato para credits  int64


In [27]:
# Se cambia el tipo de dato de los id en el cataset de movies, con le fin de trabajar con el mismo tipo de dato en el merge de dataset.
df['id'] = df['id'].astype(int)
print('tipo de dato para movies', df['id'].dtype, 'y el tipo de dato para credits ',df_c['id'].dtype)

tipo de dato para movies int32 y el tipo de dato para credits  int64


In [28]:
print(" El dataset movies tiene", df.shape[0],"filas y el dataset credist tiene", df_c.shape[0],"filas. ")

 El dataset movies tiene 45363 filas y el dataset credist tiene 45439 filas. 


In [29]:
print(df_c.duplicated(subset='id', keep='first' ))

new_duplicate = df_c[df_c.duplicated('id')]

print("id que se repiten mas de una vez:")
print(new_duplicate)

0        False
1        False
2        False
3        False
4        False
         ...  
45471    False
45472    False
45473    False
45474    False
45475    False
Length: 45439, dtype: bool
id que se repiten mas de una vez:
                                                    cast  \
23533  [{'cast_id': 3, 'character': 'Camille Claudel'...   
26638  [{'cast_id': 1, 'character': 'Gerard Carriere'...   
30013  [{'cast_id': 4, 'character': 'Mihoko Nakagawa'...   
33838  [{'cast_id': 15, 'character': 'Chuck Barris', ...   
35809  [{'cast_id': 12, 'character': 'Narrator', 'cre...   
38882  [{'cast_id': 1004, 'character': 'Luke Oarum', ...   
40051  [{'cast_id': 1, 'character': 'Jenjira', 'credi...   

                                                    crew      id  
23533  [{'credit_id': '577ed5389251416976004432', 'de...  110428  
26638  [{'credit_id': '52fe47bfc3a368484e0d77bf', 'de...   69234  
30013  [{'credit_id': '56365ed9925141285701b06e', 'de...  132641  
33838  [{'credit_id': '52

In [30]:
df_c[df_c['id'] == 110428] # en este ejemplo vemos que el mismo id tiene valores distintos en el campo crew.
# Esta es una razón de que porque el dataset credits tiene mas registros que movies.-

,cast,crew,id
4356,"[{'cast_id': 3, 'character': 'Camille Claudel'...","[{'credit_id': '52fe4ad6c3a36847f81e461b', 'de...",110428
23533,"[{'cast_id': 3, 'character': 'Camille Claudel'...","[{'credit_id': '577ed5389251416976004432', 'de...",110428


In [30]:
df_elenco = pd.DataFrame(df_c['cast'], columns = ['cast_id', 'character', 'credit_id','gender', 'id', 'name', 'order','profile_path'])

In [32]:
df_elenco.head()

,cast_id,character,credit_id,gender,id,name,order,profile_path


In [32]:
# se realiza un merge de los dataset para el calculo de las ultimas funciones en la api get_user y get_director
result = pd.merge(df, df_c, on="id", how="inner")

In [33]:
result.to_csv('../movies/dataset/dataset_final.csv', index=False)

In [37]:
# se crea un dataset reducido a 25 mb para utilizarlo en el consumo de la api en render
# se elimina columnas que no usamos.
df = df.drop(columns = ['belongs_to_collection','production_companies','production_countries', 'original_language'])

df

# df.head()

,budget,genres,id,overview,popularity,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,release_year,release_month,release_day_of_week,retorno
0,30000000.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,7.7,5415.0,1995,10,0,12.451801
1,65000000.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,1995,12,4,4.043035
2,0.0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,A family wedding reignites the ancient feud be...,11.7129,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,1995,12,4,NaN
3,16000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,"Cheated on, mistreated and stepped on, the wom...",3.859495,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,1995,12,4,5.090760
4,0.0,"[{'id': 35, 'name': 'Comedy'}]",11862,Just when George Banks has recovered from his ...,8.387519,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0,1995,2,4,inf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45460,0.0,"[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...",30840,"Yet another version of the classic epic, with ...",5.683753,1991-05-13,0.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Robin Hood,5.7,26.0,1991,5,0,NaN
45462,0.0,"[{'id': 18, 'name': 'Drama'}]",111109,An artist struggles to finish his work while a...,0.178241,2011-11-17,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,9.0,3.0,2011,11,3,NaN
45463,0.0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",67758,"When one of her hits goes wrong, a professiona...",0.903007,2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,3.8,6.0,2003,8,4,NaN
45464,0.0,[],227506,"In a small town live two brothers, one a minis...",0.003503,1917-10-21,0.0,87.0,[],Released,NaN,Satan Triumphant,0.0,0.0,1917,10,6,NaN


In [ ]:
df.to_csv('../movies/dataset/dataset_reducido.csv', index=False)